在`Ruby`中，类的定义有所不同，使用`class`关键字时。你不仅仅在规定对象的行为方式，你实际上在运行代码。

类只不过是一个增强的模块；因此本章学到的所有只是也都可以应用于模块。

你可能认为定义类就是定义方法。其实，你可以在类定义中放入任何代码

In [1]:
def MyClass
  puts 'Hello'
end

:MyClass

In [2]:
result = class MyClass
  self
end

MyClass

In [3]:
def testt
end

:testt

在程序的顶层，当前类是`Object`，这是`main`对象所属的类（这就是你在顶层定义方法会称为`Object`实例方法的原因。）

In [4]:
p self.class

Object


Object

In [5]:
p self

main


main

In [6]:
self.class

Object

In [7]:
self.private_methods

[:include, :using, :public, :private, :define_method, :DelegateClass, :Digest, :MyClass, :testt, :sprintf, :format, :Integer, :Float, :String, :Array, :Hash, :fail, :iterator?, :__method__, :catch, :__dir__, :loop, :global_variables, :throw, :block_given?, :raise, :binding, :URI, :__callee__, :eval, :trace_var, :untrace_var, :at_exit, :Rational, :Complex, :select, :set_trace_func, :gem, :caller, :caller_locations, :`, :test, :fork, :exit, :Pathname, :sleep, :respond_to_missing?, :load, :gem_original_require, :syscall, :open, :printf, :print, :putc, :puts, :gets, :readlines, :readline, :initialize_copy, :initialize_clone, :initialize_dup, :system, :p, :exec, :exit!, :proc, :lambda, :srand, :rand, :spawn, :abort, :pp, :trap, :require, :require_relative, :autoload, :autoload?, :local_variables, :warn, :method_missing, :singleton_method_added, :singleton_method_removed, :singleton_method_undefined, :initialize]

In [8]:
class C
  def m1
    def m2; end
  end
end

class D < C; end

obj = D.new
obj.m1

C.instance_methods(false)

[:m1, :m2]

在一个方法中，当前类就是当前对象的类。（试着在一个方法中用`def`关键字定义另外一个方法，你会发现这个新方法会定义在`self`所属的类中）

`Module#class_eval`方法会在一个已存在类的上下文中执行一个块

In [9]:
def add_method_to(a_class)
  a_class.class_eval do
    def m; 'Hello!';end
  end
end

add_method_to String
"abc".m

"Hello!"

`Module#class_eval`方法和`Object#instance_eval`方法截然不同。`instance_eval`方法只修改`self`，而`class_eval`方法会同时修改`self`和当前类

`Module#class_eval`方法实际上比`class`关键字更加灵活。可以对任何代表类的变量使用`class_eval`方法，而`class`关键字只能使用常量。另外,`class`关键字会打开一个新的作用域，这样将丧失当前绑定的可见性。而`class_eval`方法则使用**扁平作用域**

`instance_eval`和`class_eval`方法该如何选择呢？通常我们用`instance_eval`打开非类的对象；而用`class_eval`打开类定义，然后用`def`定义方法。如果要打开的对象也是类，那该用`instance_eval`还是`class_eval`呢？

如果你只想修改`self`，那么`instance_eval`方法和`class_eval`方法都可以出色的完成任务。不过，你应该选择更能准确表达你意图的方法。如果你希望打开一个对象，但并不关心它是不是一个类，那么`instance_eval`就很很好。如果你想使用打开类技巧来修改类，那么`class_eval`方法显然是更好的选择。

Ruby解释器假定所有的实例变量都属于当前对象`self`。在类定义时也如此。

In [10]:
class MyClass
  @my_val = 1
end

1

In [11]:
class MyClass
  @my_val = 1
  def self.read; @my_var; end
  def write; @my_var = 2; end
  def read; @my_var; end
end

:read

In [12]:
obj = MyClass.new
p obj.read
p obj.write
p obj.read
p MyClass.read

nil
2
2
nil


上面的程序定义了两个实例变量，它们正好都叫`@my_var`，但是它们分属不同的作用域。要清楚怎么回事，需要牢记类也是对象。

## 类变量

In [13]:
class C
  @@v = 1
end

1

In [14]:
class D < C
  def my_method; @@v; end
end

:my_method

类变量和类实例变量不同，它们可以被子类或者类的实例所使用。

In [15]:
D.new.my_method

1

不幸的是，类变量有一个不好的怪癖，下面是一个例子：

In [16]:
@@v = 1

class MyClass
  @@v = 2
end

@@v

(pry):54: warning: class variable access from toplevel
(pry):60: warning: class variable access from toplevel


2

得到这样的结果是因为类变量并不真正属于类，它属于类体系结构。由于`@@v`定义于`main`的上下文，它属于`main`的类`Object`，所以也属于`Object`所有的后代。

In [17]:
c = Class.new(Array) do
  def my_method
    'Hello!'
  end
end

#<Class:0x1aafd20>

In [18]:
c.name

In [19]:
MyClass = c
MyClass.name

(pry):67: warning: already initialized constant MyClass
(pry):4: warning: previous definition of MyClass was here


"MyClass"

有趣的是，`Ruby`在这里耍了一个小把戏。当你把一个匿名类赋值给一个变量时，`Ruby`知道你是想给这个类命名，他会对这个类说，这是你的新名字

In [20]:
str = "just a regular string"

def str.title?
  self.upcase == self
end

p str.title?
p str.methods.grep(/title?/)
p str.singleton_methods

false
[:title?]
[:title?]


[:title?]

这段代码为`str`添加了一个`title?`方法，其他对象（即使是`String`对象）没有这个方法。只对单个对象生效的方法，称为单件方法。你可以用上面的语法定义单件方法，也可以用`Object#define_singleton_method`来定义。

In [21]:
paragraph = "any string can be a paragraph"

def paragraph.title?
  self.upcase == self
end

index(paragraph)

NoMethodError: undefined method `index' for main:Object

我们知道类也是对象，而类名只是常量

**类方法的实质是，它是一个类的单件方法。**

向对象询问它的类时，`Ruby`并没有告诉你全部的真相。你得到的类并非你看到的类，而是一个对象特有的隐藏类。这个类被称为该对象的单件类，也有人称为元类。

像`Object#class`这样的方法会小心翼翼地把单间类隐藏起来，但是你可以用与会战术解决这个问题。

In [22]:
obj = Object.new

singleton_class = class << obj
  self
end

singleton_class.class

Class

In [23]:
"abc".singleton_class

#<Class:#<String:0x1bc2568>>

In [24]:
obj.class

Object

上面的例子还说明单间类也是类，是特殊的类。对于初学者来说，它一直是不可见的，除非用`Object#singleton_class`方法或`class<<`语法获得它。**同时，每个单间类只有一个实例，而且不能被继承，更重要的是，单件类是一个对象的单件方法的存活之所。**

In [25]:
def obj.my_singleton_method; end
singleton_class.instance_methods.grep(/my_/)

[:my_singleton_method]

In [26]:
class C
  def a_method
    'C#a_method()'
  end
end

class D < C; end
obj = D.new
obj.a_method

"C#a_method()"

Ruby 有一种特殊的基于单件类的引用，可以在离开该作用域时返回`self`。

In [27]:
class << obj
  def a_singleton_method
    'obj#a_singleton_method()'
  end
end

:a_singleton_method

In [28]:
obj.singleton_class.superclass

D

现在我们知道如何查找单件方法了。如果对象有单件类，`Ruby`不是从他所在的类开始找，而是从对象的单件类中开始查找方法。

现在我们为“小白鼠”程序加上一个类方法。

In [29]:
class C
  class << self
    def a_class_method
      'C.a_class_method()'
    end
  end
end


:a_class_method

In [30]:
C.singleton_class

#<Class:C>

In [31]:
D.singleton_class

#<Class:D>

In [32]:
D.singleton_class.superclass

#<Class:C>

In [33]:
C.singleton_class.superclass

#<Class:Object>

类方法只是特殊的单件方法

单件类的超类就是超类的单件类

这种方式，可以让你在子类中调用父类的类方法

In [34]:
D.a_class_method

"C.a_class_method()"

Ruby对象模型一共有七条规则

- 只有一种对象—— 要么是普通对象，要么是模块
- 只有一种模块——可以是一个普通模块，一个类或者一个单件类
- 只有一种方法，它存在于一个模块中——通常是在一个类中
- 每个对象（包括类）都有自己的“真正的类”——要么是一个普通类，要么是一个单件类。
- 除了`BasicObject`类没有超类外，每个类有且都有一个祖先 —— 要么是一个类，要么是一个模块。这以为着任何类只有一条向上的、直到`BasicObject`的祖先链。
- 一个对象单件类的超类是这个对象的类，一个类的单件类的超类是这个类的超类的单间类。
- 调用一个方法时，Ruby先右迈一步进入接收者真正的类，然后向上刚进入祖先链，这是ruby查找方法的方式。

有三种方法来定义类方法

In [35]:
def MyClass.a_class_method;end

:a_class_method

In [36]:
class MyClass
  def self.another_class_method;end
end

:another_class_method

In [37]:
class MyClass
  class << self
    def yet_another_class_method;end
  end
end

:yet_another_class_method

`instance_eval`会修改`self`，而`class_eval`会对`self`和当前类都进行修改。其实`instance_eval`方法也会修改当前类；它会把当前类修改为接收者的单件类。下面的例子使用了`instance_eval`方法来定义一个单件方法。

In [38]:
s1, s2 = "abc", "def"

s1.instance_eval do
  def swoosh!; reverse; end
end

s1.swoosh!
s2.respond_to?(:swoosh!)

false

`instance_eval`的标准含义是是：我想修改`self`。

如果想给类创建属性呢？你也许会试图重新打开`Class`，然后在那里定义属性

In [39]:
class MyClass; end

class Class
  attr_accessor :b
end

MyClass.b = 42
MyClass.b

42

In [40]:
class MyClass
  class << self
    attr_accessor :c
  end
end

虽然这样做可行，但它会给所有的类都添加这个属性。如果希望添加只属于`MyClass`的属性，就需要使用另一种方式。应该在它的单件类中定义这属性。

In [41]:
MyClass.c = 'It words!'
MyClass.c

"It words!"

`#BasicObject`的超类不是别人，正式我们的老朋友`Class`。

当以一个类包含一个模块时，它获得的是该模块的实例方法——而不是类方法。解决方法是：

In [42]:
module MyModule
  def my_method; 'hello'; end
end

class MyClass
  class << self
    include MyModule
  end
end

MyClass.my_method

"hello"

`my_mehtod`方法是`MyClass`单间类的一个实例方法，这样，`my_method`方法也是`MyClass`的一个类方法。这种技巧称为类扩展

In [43]:
module MyModule
  def my_method; 'hello'; end
end

obj = Object.new

class << obj
  include MyModule
end

obj.my_method
obj.singleton_methods

[:my_method]

类扩展通过向类的单件类中添加模块来定义类方法。类方法其实是单件方法的特里，因此你可以把这种技巧用到任意对象上。

`Object#extend`方法其实是在接收者的单件类中包含模块的快捷方式。

In [44]:
module Kernel
  alias_method :require_without_record, :require
  
  def require(file)
    $require << file if caller[1] =~ /rake2thor:/
    require_without_record file
  end
end

:require

可以通过如下散步来编写**环绕别名**：

1. 给方法定义一个别名
2. 重定义这个方法
3. 在新的方法中调用老的方法

环绕别名的一个缺点在于它污染了你的类，为它添加了一个额外的名字。要解决这个小问题，可以在添加别名之后，想办法把老版本的方法变成私有的

我们已经知道**细化**可以把一段代码直接加入一个类中。细化还有一个额外的功能，可以用来替换环绕别名。

In [45]:
module StringRefinement
  refine String do
    def length
      super > 5 ? 'long' : 'short'
    end
  end
end

#<refinement:String@StringRefinement>

In [46]:
using StringRefinement
"war and Peace".length

"long"

上面的代码细化了`String`类，它为`length`函数包装了额外的功能。像其他细化技术一样。细化封装器的作用范围只到文件末尾处。这让细化封装器比等价的环绕别名方法更安全，因为环绕别名是全局性的。

In [47]:
module ExplicitString
  def length
    super > 5 ? 'long' : 'short'
  end
end

String.class_eval do
  prepend ExplicitString
end

String

这意味着`prepend`方法包含的模块可以复写该类的同名方法，同时可以通过`super`调用该类中的原始方法。

In [49]:
"war and Peace".length

ArgumentError: comparison of String with 5 failed

可以称这种技术叫**下包含包装器**。与细化包装器相比，它不是一种局部化的技巧，但是一般认为它比细化封装器和环绕别名更加清晰。

使用环绕别名使 `1 + 1 = 3`